In [ ]:
# Google Colab 시스템 실행 방법

import sys
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*

## **분류 (Classification)**

**주제 : 헌혈 여부 예측 🚑**

대만의 이동식 헌혈 차량 헌혈자들의 정보를 담은 데이터셋을 다룹니다. 대만의 수혈 서비스 센터 (The Blood Transfusion Service Center)는 이동식 차량으로 대학교들을 돌면서 헌혈 행사를 진행, 수혈을 위한 혈액을 기부 받습니다.

분류모델의 목표는 각 헌혈자의 정보를 활용해서 **헌혈자가 2007년 3월에 헌혈을 했는지 여부**를 예측하는 것입니다. 

헌혈 여부 및 공급 요구를 추적하고 예측하는 좋은 데이터 기반 시스템은 **공급망 전체를 개선하여 더 많은 환자가 필요한 수혈을 받도록** 할 수 있다는 기대효과가 있습니다.

In [ ]:
# 분석에 필요한 라이브러리 및 데이터셋을 불러옵니다
import pandas as pd

donors = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')
assert donors.shape == (748,5)  # 데이터셋이 잘 불러와졌는지 assert를 사용해 검증합니다.

# 칼럼의 이름을 이해하기 쉽도록 변경합니다.
donors = donors.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [ ]:
donors.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


### **컬럼 설명**

- **months_since_last_donation**: 마지막으로 헌혈을 한 후에 지난 개월수
- **number_of_donations**: 헌혈 횟수
- **total_volume_donated**: 기부한 혈액 총량
- **months_since_first_donation**: 첫 헌혈을 한 후에 지난 개월수
- **made_donation_in_march_2007**: 2007년 헌혈을 했는지 여부 (target)

데이터를 본다면 약 3/4에 해당하는 과반수가 2007년 3월에 헌혈을 하지 않았다는 것을 알 수 있습니다.  
아래는 Baseline 모델의 정확도 점수 (accuracy score)를 나타낸 것입니다.

In [ ]:
#0(헌혈을 하지 않은 사람)과 1(헌혈을 한 사람)의 비율을 살핍니다.
donors['made_donation_in_march_2007'].value_counts(normalize=True)

0    0.762032
1    0.237968
Name: made_donation_in_march_2007, dtype: float64

### **데이터를 feature (X), label(y)로 분리해 train/test 데이터셋으로 무작위 분할**

In [ ]:
# 피쳐값과 타겟값을 분류해준 뒤, 80%는 피쳐값으로, 20%는 타겟값으로 설정합니다.
from sklearn.model_selection import train_test_split
X_df = donors.drop(['made_donation_in_march_2007'], axis=1)
y_df = donors['made_donation_in_march_2007']

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2)

### **Logistic Regression**

**1. MAE 값을 최소로 하는 feature 수 & 종류를 알아보며 feature 엔지니어링을 진행**


**2. Scikit-Learn 으로 Logistic Regression 모델링 및 학습(fit)**

---




In [ ]:
# MAE 값을 최소로 하는 feature 수 & 종류를 알아보며 feature 엔지니어링을 진행합니다.

from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

for i in range(1,len(X_df.columns)+1):
  selector = SelectKBest(score_func=f_regression, k=i)
  X_train_sel = selector.fit_transform(X_train, y_train)
  X_test_sel = selector.transform(X_test)

  lr.fit(X_train_sel, y_train)
  pred = lr.predict(X_test_sel)
  mae = mean_absolute_error(pred, y_test)
  print(i, round(mae,1), X_df.columns[selector.get_support()] )

1 0.2 Index(['months_since_last_donation'], dtype='object')
2 0.2 Index(['months_since_last_donation', 'number_of_donations'], dtype='object')
3 0.2 Index(['months_since_last_donation', 'number_of_donations',
       'total_volume_donated'],
      dtype='object')
4 0.2 Index(['months_since_last_donation', 'number_of_donations',
       'total_volume_donated', 'months_since_first_donation'],
      dtype='object')


In [ ]:
# MAE 값이 feature 의 수 차이에 큰 영향이 없으므로 전체 feature를 선택합니다.
lr.fit(X_train, y_train)
pred = lr.predict(X_test)

**정확도 산출**

In [ ]:
# accuracy score를 활용해 분류모델의 정확도를 확인합니다.
from sklearn.metrics import accuracy_score

accuracy_score(pred, y_test)

0.7733333333333333